In [1]:
%pylab inline
#from sympy import *
from scipy.integrate import *

Populating the interactive namespace from numpy and matplotlib


In [2]:
H_0 = 70.0 # km/s.Mpc

# Exercise 

for a cosmology with $\Omega_0 + \Lambda_0 = 1.0$

find $\Delta_{coll}(a_{coll}, \Omega_0, \Delta_i(coll))$

for different values of $0.1<\Omega_i<1.0$

and $a_{coll} = 1$

take $z_i = 1000$

We know that: 

$\Delta_{coll} = \dfrac{\Omega(a_{coll})}{s^3_{coll}} \left (\dfrac{a_{coll}}{a_i} \right )^3 (1 + \Delta_i)$

Using this equation we can find $\Delta_{coll}$. 

Where $a_{i} = \dfrac{1}{1 + z_{i}} = \dfrac{1}{1+1000} = 0.001$

and $s_{coll} =  F s_{ta}$

where $s_{ta} = \dfrac{2}{\sqrt{3}} \left( \dfrac{-b_2}{b_1}\right)^{1/2} cos \left(  \dfrac{\phi - 2\pi}{3} \right)$

where $\phi = arccos [x/(x^2 + y^2)^{1/2}]$ , $x = -9b_1^{1/2}$

$\int \limits_{0}^{a_{coll}} f(a) da = 2 \dfrac{H_0}{H_i}\int \limits_{0}^{s_{ta} } g(s)ds$

In [ ]:
Participantes=["Guido", "Abigail", "Veronica"]
for i in range(20):
    print random.randint(0, 3)    
print "Y el ganador del viaje a Villa de Leyva es:" ,Participantes[random.randint(0, 3)]

In [11]:
def sta(Deltai, Omegai):
    Li = 1-Omegai
    b1 = Li
    b2 = 1 - Omegai*(1+ Deltai) - Li
    b3 = Omegai*(1 + Deltai)
    x = -9*b1**0.5*b3
    y = (3*(-4*b2**3 - 27*b1))
    Phi= arccos(x/(x*x+y*y)**0.5)
    FOO = (2.0/sqrt(3)) * (-b2/b1)**0.5 *cos((Phi-2.*pi)/3.)
    return FOO

def f(a, Omega0):
    L0 = 1 - Omega0
    f = ( 1 + Omega0*(1/a  - 1) + L0*(a**2-1) )**(-0.5)
    return f

def Int_fa(Omega0):
    I = quad(f, 0, 1, args=(Omega0))
    return I

def H(z, Omega0):
    Lambda0 = 1. - Omega0
    return H_0*(Omega0*(1+z)**3 - (Omega0+Lambda0-1)*(1+z)**2 + Lambda0)**0.5

def Omega_z(z,Omega0):
    return Omega0 * (1+z)**3 * (H_0/H(z,Omega0))**2

def Lambda_z(z, Omega0):
    Lambda0=1.-Omega0
    return Lambda0*(H_0/H(z,Omega0))**2

def g_s(s,z,Omega0, Deltai):
    Omegai=Omega_z(z, Omega0)
    Lambdai=Lambda_z(z, Omega0)
    return (1+ Omegai*(1/s-1)*(1+Deltai)+Lambdai*(s*s - 1))**-0.5

def deltaicrit(Omegai):
    Lambdai = 1-Omegai
    q = (Lambdai*(8-Lambdai**2 + 20*Lambdai + 8*(1 - Lambdai)**(3/2.)))**(1/3.)
    p = 1 + 5/4. * Lambdai + 3*Lambdai*(8+Lambdai)/4*q + 3*q / 4.
    return 1/Omegai*p - 1

In [5]:
llblb = Omega_z(1000, 0.01)
deltaicrit(0.9)

2.043812857342479

In [18]:
def Int_Sta_deltai(z, Deltai, Omega0):
    Omegai = Omega_z(z, Omega0)
    
    Integral = quad( g_s, 0.00000, sta(Deltai,Omegai), args=(z,Omega0,Deltai))
    return Integral
                    

In [19]:
Int_Sta_deltai(1000,0.0001,0.7)

# La cota inferior de Deltai está dada por deltaicrit (ec 8 en el paper)

(3132.7388134820785, 1.627677193027921e-08)

In [20]:
Des = linspace(deltaicrit(0.9) ,deltaicrit(0.9)*100  ,20)

for Del in Des:
    print Del, Int_Sta_deltai(1000,Del,0.7)[0]


2.04381285734 0.0222422330553
12.6931535351 0.00010315121092
23.3424942128 1.86496048387e-05
33.9918348905 6.54673925142e-06
44.6411755683 3.07254506752e-06
55.290516246 1.69868451051e-06
65.9398569237 1.04346040983e-06
76.5891976015 6.89850679143e-07
87.2385382792 4.81475268681e-07
97.8878789569 3.50327649654e-07
108.537219635 2.63451748858e-07
119.186560312 2.0349726834e-07
129.83590099 1.60711884491e-07
140.485241668 1.29313692466e-07
151.134582346 1.05720599779e-07
161.783923023 8.76290787166e-08
172.433263701 7.35099965348e-08
183.082604379 6.2320024514e-08
193.731945057 5.33302223078e-08
204.381285734 4.60199358472e-08


#Falta 

1. definir deltaicrit como el limite inferior en la biseccion

2. ¿poner limite superior 100? ¿10 veces? ¿1000 veces?

3. Ejecutar la biseccion

In [21]:
Omega0 = 0.9
zi = 1000
Omegai = Omega_z(1000, Omega0)

A = deltaicrit(Omegai)    # s lower limit
B = 100.0*deltaicrit(Omegai)    # s upper limit
x = (A+B)/2

# bisection method
epsilon=1e-9
while (abs(
           Int_Sta_deltai( zi, x, Omega0)[0]*H(zi,Omega0)/H_0 - Int_fa(Omega0)
           )>epsilon):

    #if (f(x)*f(A) > 0  ):
    if(
        Int_Sta_deltai( zi, x, Omega0)[0]*H(zi,Omega0)/H_0 - Int_fa(Omega0) *
        Int_Sta_deltai( zi, A, Omega0)[0]*H(zi,Omega0)/H_0 - Int_fa(Omega0)
        > 0     
        ):
        A = x
        x = (A+B)/2
    else:
        B = x
        x = (A+B)/2
x_0 =x
print x

ValueError: negative number cannot be raised to a fractional power